In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql.functions import mean, col, desc, count, isnull, isnan, when, rank, sum
import pandas as pd
import matplotlib.pyplot as plt


spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.ui.showConsoleProgress", "false") \
    .getOrCreate()
    # .config("spark.master", "spark://127.0.0.1:7077") \


spark.sparkContext.setLogLevel(logLevel='ERROR')


csv_dir = 'uber_test_split'
uber = spark.read.load(csv_dir, format="csv", header="false", inferSchema="true")


In [7]:
uber = uber.withColumnRenamed("_c0", "Date/Time")
uber = uber.withColumnRenamed("_c1", "Lat")
uber = uber.withColumnRenamed("_c2", "Lon")
uber = uber.withColumnRenamed("_c3", "Base")
uber.show()

+----------------+-------+--------+------+
|       Date/Time|    Lat|     Lon|  Base|
+----------------+-------+--------+------+
|8/1/2014 0:00:00|40.6754| -74.017|B02682|
|8/1/2014 0:01:00|40.7073|-74.0054|B02598|
|8/1/2014 0:01:00|40.7633|-73.9891|B02617|
|8/1/2014 0:08:00|40.7577|-73.9715|B02617|
|8/1/2014 0:09:00| 40.726|-73.9918|B02512|
|8/1/2014 0:12:00|40.7209|-74.0507|B02512|
|8/1/2014 0:15:00|40.7223|-73.9887|B02598|
|8/1/2014 0:16:00|40.6762|-73.9708|B02682|
|8/1/2014 0:19:00| 40.722|-73.9961|B02682|
|8/1/2014 0:20:00|40.7643|-73.9983|B02598|
|8/1/2014 0:23:00|40.6858|-73.9805|B02598|
|8/1/2014 0:24:00|40.6907| -73.992|B02617|
|8/1/2014 0:27:00|40.7298|-73.9806|B02617|
|8/1/2014 0:28:00| 40.716|-73.9578|B02617|
|8/1/2014 0:28:00|  40.72|-73.9992|B02617|
|8/1/2014 0:29:00|40.7395|-74.0019|B02682|
|8/1/2014 0:29:00| 40.755|-73.9839|B02617|
|8/1/2014 0:29:00|40.7553|-73.9699|B02617|
|8/1/2014 0:39:00|40.7668|-73.9789|B02682|
|8/1/2014 0:41:00|40.7319|-74.0033|B02617|
+----------

In [10]:
print(uber.count())
print(uber.distinct().count())
print(uber.select('Date/Time', 'Lat', 'Lon', 'Base').distinct().count())
print(uber.select('Date/Time', 'Lat', 'Lon').distinct().count())
print(uber.select('Base', 'Lat', 'Lon').distinct().count())
print(uber.select('Date/Time', 'Base').distinct().count())


162613
162613
162613
162606
136568
93827


In [17]:
uber.write.jdbc("jdbc:cassandra://127.0.0.1:9042/uber", "aug2014")
# https://stackoverflow.com/questions/30983982/how-to-use-jdbc-source-to-write-and-read-data-in-pyspark
# https://stackoverflow.com/questions/66675003/java-sql-sqlexception-no-suitable-driver-when-tryingt-to-run-a-python-script


Py4JJavaError: An error occurred while calling o150.jdbc.
: java.sql.SQLException: No suitable driver
	at java.sql/java.sql.DriverManager.getDriver(DriverManager.java:298)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$2(JDBCOptions.scala:107)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:107)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:229)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:233)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:47)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:47)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.

In [16]:
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])
session = cluster.connect()
session.execute("CREATE KEYSPACE IF NOT EXISTS uber WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 1};")
session.shutdown()
session = cluster.connect('uber')

# prepared = session.prepare("INSERT INTO uber.2014 (id, name) VALUES (?, ?)")
# session.execute(prepared, (1, 'John'))


In [26]:
import uuid
from cassandra.cqlengine import columns, connection
from cassandra.cqlengine.management import sync_table
from cassandra.cqlengine.models import Model

class Aug2014(Model):
    id          = columns.TimeUUID(primary_key=True, default=uuid.uuid1)
    date_time   = columns.DateTime()
    lat         = columns.Double()
    lon         = columns.Double()
    base        = columns.Text()
    prediction  = columns.SmallInt(index=True)

connection.setup(['127.0.0.1'], "uber", protocol_version=3)
sync_table(Aug2014)
